# Modifying a Fleur input file

## Table of contents
<ul class="toc-item">
    <li><a href="#Modifying-a-Fleur-input-file">Modifying a Fleur input file</a>
        <ul class="toc-item">
            <li><a href="#Task---1">Task - 1</a></li>
            <li><a href="#FleurinpModifier">FleurinpModifier</a>
                <ul class="toc-item">
                    <li><a href="#Invalid-changes">Invalid changes</a></li>
                    <li><a href="#Methods-for-species-manipulation">Methods for species manipulation</a></li>
                    <li><a href="#Store-to-the-database">Store to the database</a></li>
                </ul>
            </li>
            <li><a href="#XML-methods">XML methods</a></li>
            <li><a href="#Task---2">Task - 2</a></li>
        </ul>
    </li>
</ul>

Some usefull imports

In [ ]:
import os
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

In [ ]:
from aiida.orm import load_node
from aiida.orm import DataFactory

## Task - 1

As discussed in the previous notebook create a `FleurinpData` node named `fleurinp` and proceed to the next section. If you have been reading things carefully you should be able to do it in one line.
<!-- fleurinp = load_node(xxx) -->

In [ ]:
fleurinp = ?

## FleurinpModifier

The `FleurinpData` from the previous notebook can not be modified in-place because it is sealed once it is stored in the database. Therefore we always need to create a new `FleurinpData` object to change an existing one. 

To make the changes and store the results in the database, AiiDA-FLEUR has `FluerinpModifier` class.

In [ ]:
from aiida.orm.data.fleurinp.fleurinpmodifier import FleurinpModifier

In [ ]:
fleurmode = FleurinpModifier(fleurinp)

For a start we set `itmax` to `30` and `minDistance` to `0.00002`. It can be done using `set_inpchanges` function with this every attribute that **occurs ONLY once** can be set

In [ ]:
fleurmode.set_inpchanges({'itmax': 30, 'minDistance' : 0.00002})

One can also provide a python dictionary with the parameter names and their values you like to change.

In [ ]:
change_dict = {
    'dos'       : True, 
    'ndir'      : -1, 
    'minEnergy' : -0.8,
    'maxEnergy' : 0.8, 
    'sigma'     : 0.005,
}

fleurmode.set_inpchanges(change_dict)

The changes are in stock, **not applied yet** you can see how the file would look like with the changes, also we want to check if the changes are valid

In [ ]:
fleurmode.show(validate=True)   #display=False

### Invalid changes

In [ ]:
# If the changes are not valid we will get an error
fleurmode_fail = FleurinpModifier(fleurinp)
fleurmode_fail.set_inpchanges({'itmax': -10, 'minDistance' : -10})
fleurmode_fail.show(validate=True, display=False)

In [ ]:
# or if you misstype a key wrong
fleurmode_fail.undo()
fleurmode_fail.set_inpchanges({'itma': 10, 'minDistance' : 10})
fleurmode_fail.show(validate=True, display=False)

### Methods for species manipulation

Change muffin tin radii, or any species parameters you have to parse a nested dict with the subtags

In [ ]:
fleurmode.set_species('W-1', {'mtSphere' : {'radius' : 3.5, 'gridPoints' : 841}, 
                              'atomicCutoffs' : {'lmax' : 9, 'lnonsphr' : 6}})


In [ ]:
fleurmode.show(validate=True)

In [ ]:
fleurmode.changes()

all these changes are currently only in memory, we can revert them, also if we made a mistake

In [ ]:
fleurmode.undo()
fleurmode.undo()
a = fleurmode.changes()
fleurmode.show(validate=True)

### Store to the database

With `freeze` function we store a new `fleurinpData` object with the applied changes in the database.

In [ ]:
fleurmode.freeze()

## XML methods

In [ ]:
# general methods (xml methods)
# if you know a bit about xml and xpath expressions, which can be very powerful
# fleurinp supports this with some more general methods. (better in the future)
# but you should know what you are doing :)

In [ ]:
#general xpath methods, you can also set a muffin thin radius like this
xpathn = '/fleurInput/atomSpecies/species[@name = "{}"]/mtSphere'.format('W-1')
fleurmode.xml_set_all_attribv(xpathn, 'radius', 6.66) # just for demo, the value does not make sence
fleurmode.show(validate=True)#, display=False) # confirm that this happened

In [ ]:
# also we could 'easily' rewrite the kpoints tag
nkpts = 800
fleurmode.set_nkpts(count=nkpts)
fleurmode.show(validate=True)#, display=False) # confirm that this happened
fleurmode.undo()

In [ ]:
from lxml import etree
# sure we could have done so brute fore with xml methods: 
kpoint_xpath = '/fleurInput/calculationSetup/bzIntegration/kPointList'
#kpoint_xpath = '//kPointList'
#kpoint_xpath = '/fleurInput/calculationSetup/bzIntegration/kPointCount'

nkpts = 400
gamma='F'

new_kpt_tag = etree.Element('kPointCount', count="{}".format(nkpts), gamma="{}".format(gamma))
fleurmode.replace_tag(kpoint_xpath, new_kpt_tag)
fleurmode.show(validate=True)

In [ ]:
# change atom group specific parameters
#fleurmode.set_atomgr_att('force', True, position=(0.0, 0.0, 0.0))

In [ ]:
fleurmode.undo()
fleurmode.undo()

In [ ]:
# now we apply the changes and store a new fleurinp data with a new inp.xml file in the database
changed_fleurinp= fleurmode.freeze()
print changed_fleurinp

In [ ]:
#Look at the differences
#print fleurmode._original
#out='changed_fleurinp'

## Task - 2 

check that the provenace in the database is kept (we have produced several new fleurinpdatas)
<!-- draw_parents(changed_fleurinp.pk, dist=4) -->